# SQLAlchemy

SQLAlchemy is a library that facilitates the communication between Python programs and databases. Most of the times, this library is used as an Object Relational Mapper (ORM) tool that translates Python classes to tables on relational databases and automatically converts function calls to SQL statements. SQLAlchemy provides a standard interface that allows developers to create database-agnostic code to communicate with a wide variety of database engines.

Application <-> SQLAlchemy <-> DBAPI (DataBase API) databases (MySQL, SQLite, PostgreSQL etc.)

Whenever we want to use SQLAlchemy to interact with a database, we need to create an Engine, which will perform communication with DBAPI functions. 

https://www.pythonsheets.com/notes/python-sqlalchemy.html

In [ ]:
# creating an engine for working with SQLite database
from sqlalchemy import create_engine
engine = create_engine("sqlite:///:memory:", echo=True) # echo=True will allow you to see all SQL commands

The declarative system is used to specify database tables and their corresponding Python classes. This system allows you to associate tables with classes. It uses a base class to control the interface between classes and tables.

In [ ]:
# a declarative base class is created
from sqlalchemy.ext.declarative import declarative_base
Base = declarative_base()

The SQLAlchemy library supports all popular data types. Minimum requirements for model development: \_\_tablename\_\_ and at least one column.

In [ ]:
# a declarative class is created
from sqlalchemy import Column, Integer, String

class User(Base):
    __tablename__ = "users"
    
    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)
    
    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (self.name, self.fullname, self.password)

After the User class is created through the declarative system, we described the information about the table, also called metadata. The created table object is a member of the larger collection MetaData. MetaData is a register of all table objects and it performs their creation in a database.

In [ ]:
User.__table__

In [ ]:
# a table creation in the database
Base.metadata.create_all(engine)

The Users class is now associated with the database table.

In [ ]:
# an object of the class Users is created
tom_user = User(name="tom", fullname="Tomas Jonaitis", password="tomo_password")

In [ ]:
print(tom_user.name, tom_user.fullname, tom_user.password)

After creating the desired tables, you can start "chatting" (creating, editing, deleting, searching records) with the database. This is done through a session.

In [ ]:
from sqlalchemy.orm import sessionmaker
Session = sessionmaker(bind=engine)
session = Session()

In [ ]:
# data transfer to session
session.add(tom_user)

While the data is not stored in the database, it is in a pending state. The data will be written to the table only when the commit command is called. If the data search function were called at this time, the data in the pending state would be automatically written to the database. 

The library ensures that all operations performed in a session will operate on the same data.

In [ ]:
# a search is performed in the database by the name field
our_user = session.query(User).filter_by(name="tom").first()

print(our_user)
print(tom_user is our_user)

In [ ]:
# new User objects are written to the session add_all([])
session.add_all([
    User(name="jonas", fullname="Jonas Jonaitis", password="foobar"),
    User(name="petras", fullname="Petras Petraitis", password="xxg527"),
    User(name="zigmas", fullname="Zigmas Zigmaitis", password="blah")])

In [ ]:
# the user password is changed
tom_user.password = "f8s7ccs"

The recorded data for the session can be seen:

In [ ]:
print(session.dirty)
print(session.new)
print(session.deleted)

In [ ]:
# initiating the writting data into the database
session.commit()

In [ ]:
# the data is listed in the database, the generated field values can be seen
print(tom_user.id)

# Columns

Column - a model attribute that stores one type of data, it corresponds to a column in a database table. A column has attributes that give it certain properties. Commonly used: default, autoincrement, nullable, primary_key, unique, ForeignKey. More about columns: http://docs.sqlalchemy.org/en/latest/core/metadata.html#sqlalchemy.schema.Column.

defaut - sets the default value of the column field. The value is assigned to the field if it is left blank during recording.

autoincrement - True/False, sets the "autoincrement" property of the field. Applies to Integer type columns.

nullable - True/False if the column is set to False - during recording, a check is made to ensure that the field is not empty, i.e. whether an empty value is possible or not.

primary_key - True/False if True sets the field as primary.

unique - True/False if True a check is made during insertion that the field value is unique across all column entries.

ForeignKey - a field constraint, which indicates that the column contains data from the specified column.

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, DateTime

engine = create_engine("sqlite:///:memory:", echo=True)

Base = declarative_base()

class Temporary(Base):
    __tablename__ = "temporary"
    
    id = Column(Integer, primary_key=True, autoincrement=True)
    name = Column(String(20), default='DEFAULT_VALUE')
    fullname = Column(String(30), nullable=False)
    price = Column(Numeric(10, 2), unique=True)
    created_at = Column(DateTime)
    
    def __repr__(self):
        return "<Temporary(name='%s', fullname='%s', price='%s', created_at='%s')>" % \
    (self.name, self.fullname, self.price, self.created_at)

Base.metadata.create_all(engine)

In [ ]:
from sqlalchemy.orm import sessionmaker
import datetime

Session = sessionmaker(bind=engine)
session = Session()

temp = Temporary(fullname='Jonas Petraitis', price=10.34, created_at=datetime.datetime.now())

session.add(temp)
session.commit()

In [ ]:
print(session.query(Temporary).all())

# Column types

The most commonly used column data types: Integer, Numeric, String, DateTime. Each type can have its own type-specific parameters, such as length, uniqueness etc. More about data types: http://sqlalchemy-html.readthedocs.io/en/rel_1_0_6/core/type_basics.html.

Integer
```python
field_name = Column(Integer)
```

Numeric
```python
field_name = Column(Number(precision=10, scale=2, decimal_return_scale=None, asdecimal=True))
```

String
```python
field_name = Column(String(length=32, collation=None, convert_unicode=False, unicode_error=None, _warn_on_bytestring=False))
```

DateTime
```python
field_name = Column(DateTime(timezone=False))
```

In [ ]:
from sqlalchemy import create_engine
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy import Column, Integer, String, Numeric, DateTime

engine = create_engine("sqlite:///:memory:", echo=True)

Base = declarative_base()

class Temporary(Base):
    __tablename__ = "temporary"
    
    id = Column(Integer, primary_key=True)
    name = Column(String(20))
    fullname = Column(String(30))
    price = Column(Numeric(10, 2))
    created_at = Column(DateTime)
    
    def __repr__(self):
        return "<Temporary(name='%s', fullname='%s', price='%s', created_at='%s')>" % \
    (self.name, self.fullname, self.price, self.created_at)

Base.metadata.create_all(engine)

In [ ]:
from sqlalchemy.orm import sessionmaker
import datetime

Session = sessionmaker(bind=engine)
session = Session()

temp = Temporary(id=1, name='Jonas', fullname='Jonas Petraitis', price=10.3411, created_at=datetime.datetime.now())

session.add(temp)
session.commit()

In [ ]:
print(session.query(Temporary).all())

# Rolling back

Because sessions work with transactions, there is an option to revert not commit changes.

In [ ]:
tom_user.name = "vidmantas"

fake_user = User(name="netikras", fullname="Netikras vardas", password="Netikras slaptažodis")
session.add(fake_user)

In [ ]:
# the data is listed in the transaction
print(session.query(User).filter(User.name.in_(["vidmantas", "netikras"])).all())

In [ ]:
# rolling back
session.rollback()

In [ ]:
print(session.query(User).filter(User.name.in_(["vidmantas", "netikras", "tom"])).all())
print(fake_user in session)

# Query formation

A query object is created using the Session method query().

In [ ]:
for instance in session.query(User).order_by(User.id):
    print(instance.name, instance.fullname)

In [ ]:
# the request for specific fields returns the response as a list
for name, fullname in session.query(User.name, User.fullname):
    print(name, fullname)

for row in session.query(User.name, User.fullname):
    print(row)
    
for row in session.query(User.name, User.fullname):
    print(row.name, row.fullname)
    
for row in session.query(User.name.label("name_label")).all():
    print(row.name_label)

In [ ]:
# data restriction LIMIT and OFFSET
for u in session.query(User).order_by(User.id)[1:3]:
    print(u)

In [ ]:
# filtering with filter_by(). filter_by() for generating simple queries
# by selecting them according column names using kwargs
for name in session.query(User.name).filter_by(name="tom"):
    print(name)
    
for name in session.query(User.name).filter_by(name="tom").filter_by(fullname="Tomas Jonaitis"):
    print(name)

In [ ]:
# filtering with filter(). filter() allows you to form more complex queries without using kwargs
for name in session.query(User.name).filter(User.fullname=="Tomas Jonaitis"):
    print(name)
    
for name in session.query(User.name).filter(User.fullname=="Tomas Jonaitis").filter(User.name=="tom"):
    print(name)

The most commonly used filtering operations:
1. Equal
```python
query.filter(User.name == "tom")
```
2. Not equal
```python
query.filter(User.name != "tom")
```
3. Like
```python
query.filter(User.name.like("%tom%"))
```
4. ilike (case sensitive)
```python
query.filter(User.name.ilike("%tom%"))
```
5. In
```python
query.filter(User.name.in_(["tom", "ed"]))
# can be used and query()
query.filter(User.name.in_(session.query(User.name).filter(User.name.like("%tom%"))))
```
6. Not in
```python
query.filter(~User.name.in_(["ed", "tom"]))
```
7. Is null
```python
query.filter(User.name == None)
query.filter(User.name.is_(None))
```
8. Is not null
```python
query.filter(User.name != None)
query.filter(User.name.isnot(None))
```
9. And
```python
# using and_()
from sqlalchemy import and_
query.filter(and_(User.name == "tom", User.fullname == "Tomas Jonaitis"))
# multiple queries in one filter()
query.filter(User.name == "tom", User.fullname == "Tomas Jonaitis")
# multiple filter() or filter_by() methods
query.filter(User.name == "tom").filter(User.fullname == "Tomas Jonaitis")
```
10. Or
```python
from sqlalchemy import or_
query.filter(or_(User.name == "tom", User.name == "mot"))
```
11. Match
```python
query.filter(User.name.match("tom"))
```

# Returning lists

Query() the object has many methods that immediately execute the SQL query and return the results.

In [ ]:
query = session.query(User).filter(User.name.like("%m%")).order_by(User.id)

In [ ]:
# all() returns all objects as a list
print(query.all())

In [ ]:
# first() returns the first object from the database
print(query.first())

In [ ]:
# one() fully downloads all query strings if no results are available 1 
# an error MultipleResultsFound or NoResultFound
print(query.one())

In [ ]:
# like one(), only if no entries are found returns None and does not call an error
print(query.one_or_none())

In [ ]:
# scalar() calls one() method and if no error occurs, returns the first column of the first row
print(query.scalar())

# Formation of text queries

The use of text commands allows you to form more flexible queries.

In [ ]:
# use of text in query formation
from sqlalchemy import text

for user in session.query(User).filter(text("id<224")).order_by(text("id")).all():
    print(user.name)

In [ ]:
# specifying parameters with the params command
session.query(User).filter(text("id<:value and name=:name")).params(value=224, name="tom").order_by(User.id).one()

In [ ]:
# by using text() and from_statement() a full-text SQL query can be generated
session.query(User).from_statement(text("SELECT * FROM users where name=:name")).params(name="tom").all()

In [ ]:
# when the query contains duplicate names it is difficult to analyze and form queries for this purpose 
#it is possible to link the columns described in the text with the object fields
stmt = text("SELECT name, id, fullname, password FROM users where name=:name")
stmt = stmt.columns(User.name, User.id, User.fullname, User.password)
session.query(User).from_statement(stmt).params(name="tom").all()

In [ ]:
# query fields can also be specified in a query() query
stmt = text("SELECT name, id FROM users where name=:name")
stmt = stmt.columns(User.name, User.id)
session.query(User.id, User.name).from_statement(stmt).params(name="tom").all()

# Calculation of records

In [ ]:
# in this way, the entire data request is placed in the subquery SELECT count(*) AS count_1 FROM (SELECT .....)
session.query(User).filter(User.name.like("%m%")).count()

In [ ]:
# calculation with func.count() a query is formed where only a specific field is counted
# SELECT count(users.name) AS count_1, users.name AS users_name
# FROM users GROUP BY users.name
from sqlalchemy import func
session.query(func.count(User.name), User.name).group_by(User.name).all()

In [ ]:
# simple countable queries
# SELECT count(*) FROM table
session.query(func.count("*")).select_from(User).scalar()

In [ ]:
session.query(func.count(User.id)).scalar()

# Creating relationships between tables

A new model is created, which will be linked to the User model with one to many relation, i.e. one User entry will have many addresses. ForeignKey - indicates that the values in the user_id column are related to the values in the column of another model (table). Relationship - specifies in the ORM that the Address class must be associated with the User class and vice versa.

More about relationships: http://docs.sqlalchemy.org/en/latest/orm/basic_relationships.html.

Instead back_populates can be used backref, then there is no need to create a second relation, it is created automatically http://docs.sqlalchemy.org/en/latest/orm/backref.html.

In [ ]:
from sqlalchemy import ForeignKey
from sqlalchemy.orm import relationship

class Address(Base):
    __tablename__ = "addresses"
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey("users.id"))
    
    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address

In [ ]:
User.addresses = relationship("Address", order_by=Address.id, back_populates="user")

In [ ]:
# changes are written to a database
Base.metadata.create_all(engine)

In [ ]:
# a new User entry is created, its addresses can be accessed via User.addresses
sim = User(name="simas", fullname="Simas Simavičius", password="gjffdd")
print(sim.addresses)

In [ ]:
# creation of new addresses is possible through the User object
sim.addresses = [
    Address(email_address="sim@google.com"),
    Address(email_address="sim@yahoo.com")
]

print(sim.addresses)

In [ ]:
# since the models are linked in both directions, any elements of the linked models can be accessed
print(sim.addresses[1].user)

In [ ]:
# the data is written to a database
session.add(sim)
session.commit()

In [ ]:
# when querying, user data addresses are not queried for the same query
sim = session.query(User).filter_by(name="simas").one()
print(sim)

In [ ]:
# user addresses are queried from the database only when needed
print(sim.addresses)

# Related table queries

Query.filter() can be used to select data from multiple related tables, to form simple realted table queries.

In [ ]:
# FROM users, addresses WHERE users.id = addresses.user_id
for u, a in session.query(User, Address).filter(User.id==Address.user_id).\
                                        filter(Address.email_address=="sim@google.com").\
                                        all():
    print(u)
    print(a)

In [ ]:
# FROM users, addresses WHERE users.id = addresses.user_id
for u, a in session.query(User, Address).filter(User.id==Address.user_id).\
                                        filter(User.name=="simas").\
                                        all():
    print(u)
    print(a)

In [ ]:
# FROM users JOIN addresses ON users.id = addresses.user_id
session.query(User).join(Address).filter(Address.email_address=="sim@google.com").all()

In [ ]:
# FROM users JOIN addresses ON users.id = addresses.user_id
session.query(User).join(Address).filter(User.name.like("%m%")).all()

Because there is only one foreign key between the User and Address tables, query.join() can easily merge the tables, but in case if there are no or multiple keys it is required to use a different structure.
```python
query.join(Address, User.id==Address.user_id)    # specifies which fields to join
query.join(User.addresses)                       # indicates a connection from left to right
query.join(Address, User.addresses)              # indicates the connection via the model connection
query.join("addresses")                          # same using text
```

Outer join:
```python
query.outerjoin(User.addresses)
```

### Alias

When creating queries, sometimes there are situations where the same table needs to be called multiple times. In this situation, it is necessary to use table alias that allow you to separate two identical tables. 

In [ ]:
from sqlalchemy.orm import aliased
adalias1 = aliased(Address)
adalias2 = aliased(Address)

for username, email1, email2 in session.query(User.name, adalias1.email_address, adalias2.email_address).\
                                join(adalias1, User.addresses).\
                                join(adalias2, User.addresses).\
                                filter(adalias1.email_address=="sim@google.com").\
                                filter(adalias2.email_address=="sim@yahoo.com"):
    print(username, email1, email2)

### Subqueries

An SQL query is generated to obtain information about how many addresses each user has:
```SQL
SELECT users.*, adr_count.address_count FROM users LEFT OUTER JOIN
    (SELECT user_id, count(*) AS address_count
        FROM addresses GROUP BY user_id) AS adr_count
    ON users.id=adr_count.user_id
```

A query is formed, which counts the number of addresses, and in the second query, a final response is formed.

In [ ]:
# a query is formed which counts the number of addresses
from sqlalchemy.sql import func
stmt = session.query(Address.user_id, func.count("*").\
                     label("address_count")).\
                     group_by(Address.user_id).subquery()

In [ ]:
# having a query that counts quantities can be treated as a table and used in other queries
# columns from the subquery are accessed through the 'c' attribute
for u, count in session.query(User, stmt.c.address_count).outerjoin(stmt, User.id==stmt.c.user_id).order_by(User.id):
    print(u, count)

In [ ]:
# to get the address item used aliased()
stmt = session.query(Address).\
                    filter(Address.email_address != 'j25@yahoo.com').\
                    subquery()
        
adalias = aliased(Address, stmt)

for user, address in session.query(User, adalias).\
        join(adalias, User.addresses):
    print(user)
    print(address)

## Exists command

The Exists keyword in SQL returns True if there are rows in the given expression.

In [ ]:
# exists command construcion
from sqlalchemy.sql import exists

stmt = exists().where(Address.user_id==User.id)
for name, in session.query(User.name).filter(stmt):
    print(name)

In [ ]:
# exists command alternative - any()
for name, in session.query(User.name).\
        filter(User.addresses.any()):
    print(name)

In [ ]:
# In any() command can be specified filtering criteria
for name, in session.query(User.name).\
        filter(User.addresses.any(Address.email_address.like("%google%"))):
    print(name)

In [ ]:
# has() command is alternative to any(), used only with many to one relation
session.query(Address).filter(~Address.user.has(User.name=="sim")).all()

# Deleting records

In [ ]:
# deleting records
session.delete(sim)
session.query(User).filter_by(name="sim").count()

In [ ]:
# addresses remain undeleted
session.query(Address).filter(
    Address.email_address.in_(["sim@google.com", "sim@yahoo.com"])
).count()

Only User data is deleted, but the related Address data remains undeleted and their User fields become NULL. Therefore, in order for the parent and the related records to be deleted, SQLAlchemy must specify this (cascade).

In [ ]:
session.close()
Base = declarative_base()

class User(Base):
    __tablename__ = 'users'

    id = Column(Integer, primary_key=True)
    name = Column(String)
    fullname = Column(String)
    password = Column(String)

    addresses = relationship("Address", back_populates='user',
                             cascade="all, delete, delete-orphan")

    def __repr__(self):
        return "<User(name='%s', fullname='%s', password='%s')>" % (
            self.name, self.fullname, self.password)

class Address(Base):
    __tablename__ = 'addresses'
    id = Column(Integer, primary_key=True)
    email_address = Column(String, nullable=False)
    user_id = Column(Integer, ForeignKey('users.id'))
    user = relationship("User", back_populates="addresses")

    def __repr__(self):
        return "<Address(email_address='%s')>" % self.email_address

    
sim = session.query(User).get(5)
print(sim)
del sim.addresses[1]

session.query(Address).filter(
    Address.email_address.in_(["sim@google.com", "sim@yahoo.com"])
).count()

In [ ]:
session.delete(sim)
session.query(User).filter_by(name="sim").count()

session.query(Address).filter(
    Address.email_address.in_(["sim@google.com", "sim@yahoo.com"])
).count()

# Tasks

1. Create two tables: item {name, unit price, description, balance}, and category {name}. One category has many products, one product belongs to one category.
2. Create two categories (toys, food) and create items (more than 1 for each category) and assign them to categories.
3. Select all items whose quantity is higher than the number entered by the user, and the price of the item is lower than the number entered by the user.
4. Print items in a user-entered category (or part of a category name).
5. Print the balance of items in each category.
6. Allow the user to enter an item id and a new item balance. Adjust the balance of the user-selected item.
7. Allow the user to enter a item name and delete the item.
8. Improve the developed Contacts program to use the sqlite database for data storage (create_engine('sqlite:///foo.db') - file, which contains stored data).